In [2]:
# Now combine the work of One Period and No Hodlers; have multiple periods where if we are in the high state, 
# we may keep growing next period, while in the low state, we stop growing but also add hodlers to population.

# Set U(X)=X; u(q)=((q+b)**(1-eta)-b**eta)/(1-eta) or u(q)=ln(q+b)-ln(b) if eta = 1; c(q)=q 
# So u'=(q+b)**-eta; c'=1; z(q)=(1-theta)u(q)+theta*c(q); z'=(1-theta)*(q+b)**(-eta)+theta;
# l=theta*(u'(q)-c'(q))/z'(q)
# l=theta*((q+b)**(-eta)-1)/((1-theta)*(q+b)**(-eta)+theta)=theta*(1-(q-b)**eta)/(1-theta*(1-(q+b)**eta))
import math
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from IPython.core.debugger import set_trace
file = r'C:\Users\spspi\Dropbox\Documents\Cryptocurrency Project\Simulations' #Save simulation graphs here
b = .00001 # This is part of utility function, makes it steep around 0
B = .95 # Discount rate beta
lam = .75 # Meeting rate lambda
theta = .75 # Buyer bargaining power
eta = .75 # Buyer risk aversion
gammam = 0.02 # Money inflation rate
gammac = 0 # CC inflation rate
qstar = 1 - b # Value at which u'=c'

# First, need to find steady state prices. Assume enite wealth always spent.
# phi/(phi'B)-1=(1+gammam)/B=i_m=lamb*l(phi'm'+psi'c')+lamm*l(phi'm')=lamb*l(qb)+lamm*l(gm)
# psi/(psi'B)-1=(1+gammac)/B=i_c=lamb*l(phi'm'+psi'c')=lamm*l(qb)

# Create a function that finds tomorrow's q's in stst
def ststq(alpha, Mp, Cp, gammam = gammam, gammac=gammac, lam=lam, theta=theta, eta=eta):
    lamb = lam * alpha # Chance of both type meeting
    lamm = lam * (1 - alpha) # Chance of money type meeting
    if alpha > (gammac + 1 - B) / (gammam + 1 - B): # If cc is valued, can solve these equations for q, because m=M in st st
        # Both type meetings
        qb = np.minimum(np.maximum((np.maximum((theta * B * lamb - (1 - theta) * (1 + gammac - B)) / (theta*
                                             (1 + gammac - B + B * lamb)), 0)) ** (1 / eta) - b, 0), qstar)
        # Money type meetings
        qm=np.minimum(np.maximum((np.maximum((theta * B * lamm - (1 - theta) * (gammam - gammac))
                                          / (theta * (gammam - gammac + B * lamm)), 0)) ** (1 / eta) - b, 0), qstar)
        return qb, qm
    else: # If cc not valued,then get same q in both meetings
        qm = np.minimum(np.maximum((np.maximum((theta * B * lam - (1 - theta) * (1 + gammam - B))/
                                               (theta * (1 + gammam - B + B * lam)), 0))**(1 / eta) - b, 0), qstar)
        return qm, qm

# Create a function that finds tomorrow's price in stst
def ststprices(alpha, Mp, Cp, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta):
    qb, qm=ststq(alpha, Mp, Cp, gammam, gammac, lam, theta, eta) # Find qb and qm from above
    if eta != 1: # If not ln utility
        # Tomorrow's price of money
        phip = (theta * qm + (1 - theta) * ((qm + b) ** (1 - eta) - b ** (1 - eta)) / (1 - eta)) / Mp
        # Tomorrow's price of cc
        psip = (theta * (qb - qm) + (1 - theta) * ((qb + b) ** (1 - eta) - (qm + b) ** (1 - eta)) / (1 - eta)) / Cp
    else: # if ln utility
        # Tomorrow's price of money
        phip = (theta * qm + (1 - theta) * (np.log(qm + b) - np.log(b))) / Mp
        # Tomorrow's price of cc
        psip = (theta * (qb - qm) + (1 - theta) * (np.log(qb + b) - np.log(qm + b))) / Cp
    return phip, psip    

# Create an explicit utility function so q is never negative
def u(q, eta=eta):
    q = np.array(q)
    if q < 0: # Don't want negative consumption
        return -1e6
    elif eta != 1: # If not ln utility
        return ((q + b) ** (1 - eta) - b ** (1 - eta)) / (1 - eta)
    else: # if ln utility
        return np.log(qstar + b) - np.log(b)
    
def du(q, eta=eta):
    q = np.array(q)
    if q < 0: # If negative, no slope
        return 0
    else: 
        return (q + b) ** (-eta)
    
# Create a function that outputs z(q(w))=w for a given wealth w and q (solve by giving a w, set this equal to 0, and solve for q)
def wminz(q, w, eta=eta):
    return w - theta * q - (1 - theta) * u(q, eta)

# Create a function to take the derivative of z
def dz(q):
    q = np.array(q)
    if q < 0:
        return 0
    if q >= qstar:
        return 0
    else:
        return theta + (1 - theta) * (q + b) ** (-eta)
    
# Create a function to solve for q given w, this time with bisect (sensitive to first guess)
def qwealth(w, eta=eta):
    w = np.array(w)
    if w > 0:
        q = optimize.brentq(wminz, -1e25, 1e25, args=(w, eta))
        if q <= 0:
            return 0
        if q >= qstar:
            return qstar
        else:
            return q
    else:
        return 0
    
# Create a function that outputs liquidity for a given q
def liquidityq(q):
    if q >= qstar:
        return 0 
    elif q < 0:
        return 0
    else:
        return (theta * (1 - (q + b) ** eta)) / (1 - theta * (1 - (q + b) ** eta))

# Create a function to take the derivative of liquidityq
def dliqq(q):
    if q < 0:
        return 0
    if q >= qstar:
        return 0
    return - theta * eta * (q + b) ** (eta - 1) / (1 - theta * (1 - (q + b) ** eta)) ** 2

# Create a function to solve for q given w and output liquidity 
def liquidity(w):
    if w < 0:
        return 0
    return liquidityq(qwealth(w))

# Create a function to take the derivative of liquidity
def dliq(w):
    if w < 0:
        return 0
    q = qwealth(w)
    if q < 0:
        return 0
    if q >= qstar:
        return 0
    return dliqq(q) / dz(q)
    
# Create a function that outputs total surplus given wealth
def surplus(w):
    if w <= 0:
        return 0
    else:
        return u(qwealth(w)) - qwealth(w)
    
# Create a function that outputs value of continuing in stst given an alpha
def ststW(alpha, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta):
    Mp, Cp = 1, 1 # Does not matter what these are, so choose these for simplicity
    lamb = lam * alpha # Chance of both type meeting
    lamm = lam * (1 - alpha) # Chance of money type meeting
    qb, qm = ststq(alpha, Mp, Cp, gammam, gammac, lam, theta, eta) # Find qb and qm from above
    phip, psip = ststprices(alpha, Mp, Cp, gammam, gammac, lam, theta, eta) # Find prices from above
    # Can solve \bar{W}(0,0) to get              
    return - Mp * phip - Cp * psip + B / (1 - B) * theta * (lamb * (u(qb) - qb) + lamm * (u(qm) - qm)) 

In [6]:
# Define period_outcomes, which solves for agent's choices given population distribution and tomorrow's state

def period_outcomes(M, C, pi, alphal, alphah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, guessm=0, guessc=0):
    Mp, Cp = M * (1 + gammam), C * (1 + gammac) # Supplies of currencies tomorrow
    lambl = lam * alphal # Probability of meeting a both type in low state
    lamml = lam * (1 - alphal) # Probability of meeting a money type in low state
    lambh = lam * alphah # Probability of meeting a both type in high state
    lammh = lam * (1 - alphah) # Probability of meeting a money type in high state
    method = 'lm' # Method used to optimize, usually either 'hybr' or 'lm'
    pen = - 1e99 # put in a penalty if answers outside bounds
    # Next find out if c'>0 or c'=0
    def sysc0(x): # This is buyer's FOC for m (x[0]) minus hodler's FOC for m if c=0  
        mho = (Mp - mu * x[0]) / (1 - mu) # hodlers' m holdings
        cho = Cp / (1 - mu) # hodlers hold all c
        return (
            pi * phil * (1 + lam * liquidity(phil * x[0])) + (1 - pi) * phih * (1 + lam * liquidity(phih*x[0]))
            - phih * (1 + lambh * liquidity(phih * mho + psih * cho) + lammh * liquidity(phih * mho)) 
            + pen * (x[0] <= 0 or mho <= 0)
        )
    def jac0(x): # Derivative of sysc0 wrt m (x[0])
        mho = (Mp - mu * x[0]) / (1 - mu) # hodlers' m holdings
        cho = Cp / (1 - mu) # hodlers hold all c
        return np.array(
            [pi * phil ** 2 * lam * dliq(phil * x[0]) + (1 - pi) * phih ** 2 * lam * dliq(phih * x[0])
             + phih ** 2 * mu / (1 - mu) * (lambh * dliq(phih * mho + psih * cho) + lammh * dliq(phih * mho))]
        ) 
    opt1 = optimize.root(sysc0, guessm, jac=jac0, method=method, options={'factor':.1}) # Solves for m
    sig1 = opt1.success # did this converge?
    m1 = opt1.x # soultion
    # Use the condition under which buyers hold no c, if it is true, c=0
    if pi * (psih - psil) >= (pi * psil * lambl * liquidity(phil * m1)+(1 - pi) * psih * lambh * liquidity(phih * m1)
                            - psih * lambh * liquidity(phih * (Mp - mu * m1) / (1 - mu) + psih * Cp / (1 - mu))): 
        #m, c = np.minimum(m1[0], Mp / mu), 0
        m, c = m1[0], 0 # buyer's currency holdings
        mho, cho=(Mp - mu * m) / (1 - mu), (Cp - mu * c) / (1 - mu) # hodlers' currency holdings
        # Next, use w=z(q) to solve for buyer's low state and high state 
        ql, qh  = qwealth(phil * m), qwealth(phih * m)  
        # Also solve for hodler's holdings in low states
        qhobl, qhoml = qwealth(phil * mho + psil * cho), qwealth(phil * mho)
        # And high states
        qhobh, qhomh = qwealth(phih * mho + psih * cho), qwealth(phih * mho)
        # Price of money implied by buyer's FOC
        phib = B * pi * phil * (1 + lam * liquidityq(ql)) + B * (1 - pi) * phih * (1 + lam * liquidityq(qh))
        # Price of money implied by hodler's FOC
        phiho = B * phih * (1 + lambh * liquidityq(qhobh) + lammh * liquidityq(qhomh)) 
        # These should be the same, but return average
        phi = (phib + phiho) / 2 
        # Price of cc implied by hodler's FOC (only 1)
        psi = B * psih * (1 + lambh * liquidityq(qhobh))
        # Through inflation, each buyer/hodler gets the same transfer Omega
        Omega = phi * M * gammam + psi * C * gammac
        # Buyer's welfare if low state tomorrow
        Wl = Omega - phi * m + B * (phil * m + theta * lam * surplus(phil * m) + Wlp)
        # Buyer's welfare if high state tomorrow
        Wh = Omega - phi * m + B * (phih * m + theta * lam * surplus(phih * m) + Whp)
        # Buyer's expected Welfare
        W = pi * Wl + (1 - pi) * Wh
        # Hodler's welfare if low state tomorrow
        Whol = (
            Omega - phi * mho - psi * cho + B * (phil * mho + psil * cho + theta * (lambl * surplus(phil * mho + psil * cho) 
                                                                                      + lamml * surplus(phil * mho)) + Wholp)
        )
        # Hodler's welfare if high state tomorrow
        Whoh = (
            Omega - phi * mho - psi * cho + B * (phih * mho + psih * cho + theta * (lambh * surplus(phih * mho + psih * cho) 
                                                                                      + lammh * surplus(phih * mho)) + Whohp)
        )
        # Hodler's expected welfare
        Who = copy(Whoh)
        
        # Make sure hodlers' actually hold c
        if psi >= B * psih * (1 + lambh * liquidity(qhomh)): # If true, hodler also do not hold c
            def sysch0(x): # This is buyer's FOC for m (x[0]) minus hodler's FOC for m when c=0
                mho = (Mp - mu * x[0]) / (1 - mu) # hodlers' m holdings
                return (
                    pi * phil * (1 + lam * liquidity(phil * x[0])) + (1 - pi) * phih * (1 + lam * liquidity(phih * x[0]))
                    -phih * (1 + lam * liquidity(phih * mho)) 
                    + pen*(x[0] <= 0 or mho <= 0)
                )
            def jach0(x): # Derivative of sysch0 wrt m (x[0])
                mho = (Mp - mu * x[0]) / (1 - mu) # hodlers' m holdings
                return np.array(
                    [pi * phil ** 2 * lam * dliq(phil * x[0]) + (1 - pi) * phih ** 2 * lam * dliq(phih * x[0])
                     + phih ** 2 * mu / (1 - mu) * (lam * dliq(phih * mho))]
                ) 
            opt0 = optimize.root(sysch0, guessm, jac=jach0, method=method, options={'factor':.1}) # Solves for m
            sig0 = opt0.success # did this converge?
            #m = np.minimum(opt0.x[0], Mp / mu) #solution
            m = opt0.x[0] # buyer's m holdings
            mho = (Mp - mu * m) / (1 - mu) # hodlers' m holdings
            # Next, use w=z(q) to solve for buyer's and hodlers' low state and high state qs
            ql, qh, qhoml, qhomh = qwealth(phil * m), qwealth(phih * m), qwealth(phil * mho), qwealth(phih * mho) 
            # Price of money implied by buyer's FOC
            phib = B * pi * phil * (1 + lam * liquidityq(ql)) + B * (1 - pi) * phih * (1 + lam * liquidityq(qh))
            # Price of money implied by hodler's FOC
            phiho = B * phih * (1 + lam * liquidityq(qhomh)) 
            # These should be the same, but return average
            phi = (phib + phiho) / 2 
            # Through inflation, each buyer/hodler gets the same transfer Omega
            Omega = phi * M * gammam
            # Buyer's welfare if low state tomorrow
            Wl = Omega - phi * m + B * (phil * m + theta * lam * surplus(phil * m) + Wlp)
            # Buyer's welfare if high state tomorrow
            Wh = Omega - phi * m + B * (phih * m + theta * lam * surplus(phih * m) + Whp)
            # Buyer's expected Welfare
            W = pi * Wl + (1 - pi) * Wh
            # Hodler's welfare if low state tomorrow
            Whol = Omega - phi * mho + B * (phil * mho + theta * lam * surplus(phil * mho) + Wholp)
            # Hodler's welfare if high state tomorrow
            Whoh = Omega - phi * mho + B * (phih * mho + theta * lam * surplus(phih * mho) + Whohp)
            # Hodler's expected welfare
            Who = copy(Whoh)
            return (phi, phib, phiho, 0, 0, 0, m, mho, 0, 0, ql, ql, qh, qh, qhoml, qhoml, qhomh, qhomh, 
                    W, Wl, Wh, Who, Whol, Whoh, sig0)
        
        # Otherwise, hodlers hold c, and this is the solution
        return (phi, phib, phiho, psi, psi, psi, m, mho, 0, cho, ql, ql, qh, qh, qhobl, qhoml, qhobh, qhomh, 
                W, Wl, Wh, Who, Whol, Whoh, sig1)
    # If the above not true, both hold c
    def sysc(x):
        mho = (Mp - mu * x[0]) / (1 - mu) # hodler's m holdings
        cho = (Cp - mu * x[1]) / (1 - mu) # hodler's c holdings
        # This is buyer's FOC for m (x[0]) minus hodler's FOC for m
        firsteq = (
            pi * phil * (1 + lambl * liquidity(phil * x[0] + psil * x[1]) + lamml * liquidity(phil * x[0])) 
            + (1 - pi) * phih * (1 + lambh * liquidity(phih * x[0] + psih * x[1]) + lammh * liquidity(phih * x[0])) 
            - phih * (1 + lambh * liquidity(phih * mho + psih * cho) + lammh * liquidity(phih * mho))
            + pen * (x[0] <= 0 or mho <= 0 or x[1] < 0 or cho < 0)
        )
        # This is buyer's FOC for c (x[1]) minus hodler's FOC for c
        secondeq = (
            pi * psil * (1 + lambl * liquidity(phil * x[0] + psil * x[1])) 
            + (1 - pi) * psih * (1 + lambh * liquidity(phih * x[0] + psih * x[1]))
            - psih * (1 + lambh * liquidity(phih * mho + psih * cho))
            + pen * (x[0] <= 0 or mho <= 0 or x[1] < 0 or cho < 0)
        )
        return np.array([firsteq, secondeq])
    def jac(x): # Jacobian for sysc
        mho = (Mp - mu * x[0]) / (1 - mu) # hodler's m holdings
        cho = (Cp - mu * x[1]) / (1 - mu) # hodler's c holdings
        # Name's pretty descriptive, lists equation and derivative currency
        d1dm = (pi * phil ** 2 * (lambl * dliq(phil *x[0] + psil * x[1]) + lamml * dliq(phil * x[0]))
                + (1 - pi) * phih ** 2 * (lambh * dliq(phih * x[0] + psih * x[1]) + lammh * dliq(phih * x[0]))
                + phih ** 2 * mu / (1 - mu) * (lambh * dliq(phih * mho + psih * cho) + lammh * dliq(phih * mho)))
        d1dc = (pi * phil * lambl * psil * dliq(phil * x[0] + psil * x[1])
                + (1 - pi) * phih * lambh * psih * dliq(phih * x[0] + psih * x[1])
                + phih * lambh * mu / (1 - mu) * psih * dliq(phih * mho + psih * cho))
        d2dm = (pi * psil * lambl * phil * dliq(phil * x[0] + psil * x[1])
                + (1 - pi) * psih * lambh * phih * dliq(phih * x[0] + psih * x[1])
                + psih * lambh * mu / (1 - mu) * phih * dliq(phih * mho + psih * cho))
        d2dc = (pi * psil ** 2 * lambl * dliq(phil * x[0] + psil * x[1])
                + (1 - pi) * psih ** 2 * lambh * dliq(phih * x[0] + psih * x[1])
                + psih ** 2 * lambh * mu / (1 - mu) * dliq(phih * mho + psih * cho))
        return np.array([[d1dm, d1dc], [d2dm, d2dc]])
    opt2 = optimize.root(sysc, (guessm, guessc), jac=jac, method=method, options={'factor':.1}) # Solves for m and c
    sig2 = opt2.success # did this converge
    #m, c = np.minimum(opt2.x[0], Mp / mu), np.minimum(opt2.x[1], Cp / mu) #solution
    m, c = opt2.x[0], opt2.x[1]  #solution
    mho, cho = (Mp - mu * m) / (1 - mu), (Cp - mu * c) / (1 - mu)
    # Next, use w=z(q) to solve for buyer's both and money qs in the low state and high state
    qbl, qml, qbh, qmh = qwealth(phil * m + psil * c), qwealth(phil * m), qwealth(phih * m + psih * c), qwealth(phih * m)
    # Same for hodler's in low state
    qhobl, qhoml = qwealth(phil * mho + psil * cho), qwealth(phil * mho)
    # And high state 
    qhobh, qhomh = qwealth(phih * mho + psih * cho), qwealth(phih * mho)
    # Price of money implied by buyer's FOC
    phib =(B * pi * phil * (1 + lambl * liquidityq(qbl) + lamml * liquidityq(qml))
           + B * (1 - pi) * phih * (1 + lambh * liquidityq(qbh) + lammh * liquidityq(qmh)))
    # Price of money implied by hodler's FOC
    phiho = B * phih * (1 + lambh * liquidityq(qhobh) + lammh * liquidityq(qhomh))
    # These should be the same, but return average
    phi = (phib + phiho) / 2 
    # Price of cc implied by buyer's FOC
    psib = B * pi * psil * (1 + lambl * liquidityq(qbl)) + B * (1 - pi) * psih * (1 + lambh * liquidityq(qbh))
    # Price of cc implied by hodler's FOC
    psiho = B * psih * (1 + lambh * liquidityq(qhobh)) 
    # These should be the same, but return average
    psi = (psib + psiho) / 2 
    # Through inflation, each buyer/hodler gets the same transfer Omega
    Omega = phi * M * gammam + psi * C * gammac
    # Buyer's welfare if low state tomorrow
    Wl = (
        Omega - phi * m - psi * c + B * (phil * m + psil * c + theta * (lambl * surplus(phil * m + psil * c) 
                                                                                  + lamml * surplus(phil * m)) + Wlp)
    )
    # Buyer's welfare if high state tomorrow
    Wh = (
        Omega - phi * m - psi * c + B * (phih * m + psih * c + theta * (lambh * surplus(phih * m + psih * c) 
                                                                                  + lammh * surplus(phih * m)) + Whp)
    )
    # Buyer's expected Welfare
    W = pi * Wl + (1 - pi) * Wh
    # Hodler's welfare if low state tomorrow
    Whol = (
        Omega - phi * mho - psi * cho + B * (phil * mho + psil * cho + theta * (lambl * surplus(phil * mho + psil * cho) 
                                                                                  + lamml * surplus(phil * mho)) + Wholp)
    )
    # Hodler's welfare if high state tomorrow
    Whoh = (
        Omega - phi * mho - psi * cho + B * (phih * mho + psih * cho + theta * (lambh * surplus(phih * mho + psih * cho) 
                                                                                  + lammh * surplus(phih * mho)) + Whohp)
    )
    # Hodler's expected welfare
    Who = copy(Whoh)
    return (phi, phib, phiho, psi, psib, psiho, m, mho, c, cho, qbl, qml, qbh, qmh, qhobl, qhoml, qhobh, qhomh, 
            W, Wl, Wh, Who, Whol, Whoh, sig2)

In [7]:
# As seen in One Period, figures can be very sensitive to initial guesses. A good way to find an initial guess is to
# use known soltions from similar parameters. With that in mind, create good_guess, which will iterate over 
# period_outcomes starting from mu=0 so that we know c'=0. Have mu converge to truth, leave everything else the same.

def good_guess(M, C, pi, alphal, alphah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, trials):
    guessm, guessc = M / mu, 0
    mugrid = np.linspace(0, mu, trials)
    for mui in mugrid:
        (phi, phib, phiho, psi, psib, psiho, m, mho, c, cho, qbl, qml, qbh, 
         qmh, qhobl, qhoml, qhobh, qhomh, W, Wl, Wh, Who, Whol, Whoh, sig) \
        = period_outcomes(M, C, pi, alphal, alphah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mui, guessm, guessc)
        guessm, guessc = copy(m), copy(c)
    return guessm, guessc

In [16]:
# Now start a T period simulation. Record both growing and steady state values. We start in period T and work backwards.
# For now, let alphah=alphal+delta

mu=.99 # Proportion of buyers (vs hodlers)
gammam = .02 # Inflation of m
gammac = 0 # Inflation of c
pi = .1 # Chance tomorrow low state
M0 = 1 # initial money supply
C0 = 1 # initial cc supply
trials = 151 # Number of iterations for good_guess (see above cell) 
T = math.ceil(2 / pi) # maximum number of periods
t = copy(T) # period counter
alpha0 = .3 # original alpha
alphatop = .9 # highest alpha
delta = (alphatop - alpha0) / T # amount alpha may grow each period

# Number of periods
tgrid = range(T+1)
# Growing prices, price differences, and convergence
phigrid, psigrid, phidgrid, psidgrid, siggrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T), np.zeros(T), np.zeros(T)
# Steady state prices
phisgrid, psisgrid = np.zeros(T + 1), np.zeros(T + 1)
# Buyer's real balances
rbmlgrid, rbclgrid, rbmhgrid, rbchgrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1)
# Hodler's real balances
rbmholgrid, rbcholgrid, rbmhohgrid, rbchohgrid = np.zeros(T + 1), np.zeros(T + 1),np.zeros(T + 1), np.zeros(T + 1)
# steady state real balances
rbmsgrid, rbcsgrid = np.zeros(T + 1), np.zeros(T + 1)
# Buyer's and Hodlers' currency holdings
mgrid, cgrid, mhogrid, chogrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1)
# Steady state currency hodlings
msgrid, csgrid = np.zeros(T + 1), np.zeros(T + 1)
# Buyers quantity traded
qblgrid, qmlgrid, qbhgrid, qmhgrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1) 
# Hodlers quantity traded
qhoblgrid, qhomlgrid, qhobhgrid, qhomhgrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1) 
# Steady state quantity traded
qsbgrid, qsmgrid = np.zeros(T + 1), np.zeros(T + 1) 
# Buyer's expected and realized Welfare
Wgrid, Wlgrid, Whgrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1)
# Hodler's expected and realized Welfare
Whogrid, Wholgrid, Whohgrid = np.zeros(T + 1), np.zeros(T + 1), np.zeros(T + 1)
# Steady state Welfare
Wsgrid = np.zeros(T + 1)

while t >= 1: 
    if t == T: # If in highest period, prices in high state from highest st st
        # Currency supplies today and tomorrow
        M, C = M0 * (1 + gammam) ** (t - 1), C0 * (1 + gammac) ** (t - 1)
        Mp, Cp =  M * (1 + gammam), C * (1 + gammac)
        ah = copy(alphatop) # alpha in highest state
        phih, psih = ststprices(ah, Mp, Cp, gammam, gammac, lam, theta, eta) # Prices in highest state
        qbh, qmh = ststq(ah, Mp, Cp, gammam, gammac, lam, theta, eta) # Quantities in highest state
        Wh = ststW(ah, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta) # Welfare of highest state
        Whp, Whohp = copy(Wh), copy(Wh)
        # Prices and steady state prices
        phigrid[T], psigrid[T], phisgrid[T], psisgrid[T] = phih, psih, phih, psih
        # Buyer's, hodlers', and steady state quantites
        qblgrid[T], qmlgrid[T], qbhgrid[T], qmhgrid[T] = qbh, qmh, qbh, qmh
        qhoblgrid[T], qhomlgrid[T], qhobhgrid[T], qhomhgrid[T] = qbh, qmh, qbh, qmh
        qsbgrid[T], qsmgrid[T] = qbh, qmh
        # Buyer's, hodlers', and steady state real balances
        rbmlgrid[T], rbclgrid[T], rbmhgrid[T], rbchgrid[T] = Mp * phih, Cp * psih, Mp * phih, Cp * psih
        rbmholgrid[T], rbcholgrid[T], rbmhohgrid[T], rbchohgrid[T] = Mp * phih, Cp * psih, Mp * phih, Cp * psih
        rbmsgrid[T], rbcsgrid[T] = Mp * phih, Cp * psih
        # Buyer's, hodlers', and steady state currency holdings
        mgrid[T], cgrid[T], mhogrid[T], chogrid[T], msgrid[T], csgrid[T] = Mp, Cp, Mp, Cp, Mp, Cp
        # Buyer's, hodlers', and steady state Welfare
        Wgrid[T], Wlgrid[T], Whgrid[T], Whogrid[T], Wholgrid[T], Whohgrid[T], Wsgrid[T] = Wh, Wh, Wh, Wh, Wh, Wh, Wh 
        
    t += - 1 # working backwards
    al = ah - delta # alpha in lower state
    phil, psil = ststprices(al, Mp, Cp, gammam, gammac, lam, theta, eta) # Prices in lower state
    qbl, qml = ststq(al, Mp, Cp, gammam, gammac, lam, theta, eta) # Quantities in lower state
    Wlp = ststW(al, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta) # Value of lower steady state
    Wholp = copy(Wlp) # Hodler's low continuation value equal to this st st
    # Low is a steady state. If we were in this steady state, outcomes would be:
    phisgrid[t], psisgrid[t], qsbgrid[t], qsmgrid[t] = copy(phil), copy(psil), copy(qbl), copy(qml)
    rbmsgrid[t], rbcsgrid[t], msgrid[t], csgrid[t], Wsgrid[t] = M * phil, C * psil, M, C,  copy(Wlp)
    # Solve for today's prices/value
    guessm, guessc = good_guess(M, C, pi, al, ah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, trials)
    (phi, phib, phiho, psi, psib, psiho, m, mho, c, cho, qbl, qml, qbh, qmh, qhobl, qhoml, qhobh, qhomh, 
            W, Wl, Wh, Who, Whol, Whoh, sig)\
    =period_outcomes(M, C, pi, al, ah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, guessm, guessc)
    # Prices and price differences
    phigrid[t], psigrid[t], phidgrid[t], psidgrid[t] = copy(phi), copy(psi), phib - phiho, psib - psiho
    # Check if convereged
    siggrid[t] = sig
    # Buyer and Hodler quantities
    qblgrid[t], qmlgrid[t]  = qwealth(m * phil + c * psil), qwealth(m * phil) 
    qbhgrid[t], qmhgrid[t]  = qwealth(m * phih + c * psih), qwealth(m * phih)  
    qhoblgrid[t], qhomlgrid[t]  = qwealth(mho * phil + cho * psil), qwealth(mho * phil) 
    qhobhgrid[t], qhomhgrid[t]  = qwealth(mho * phih + cho * psih), qwealth(mho * phih) 
    # Buyer and Hodler real balances
    rbmlgrid[t], rbclgrid[t], rbmhgrid[t], rbchgrid[t] = m * phil, c * psil , m * phih, c * psih  
    rbmholgrid[t], rbcholgrid[t], rbmhohgrid[t], rbchohgrid[t]  = mho * phil, cho * psil, mho * phih, cho * psih 
    # Buyer and Hodler currency holdings
    mgrid[t], cgrid[t], mhogrid[t], chogrid[t] = m, c, mho, cho
    # Buyer and Hodler Welfare
    Wgrid[t], Wlgrid[t], Whgrid[t] = pi * Wl + (1 - pi) * Wh, Wl, Wh
    Whogrid[t], Wholgrid[t], Whohgrid[t] = Whoh, Whol, Whoh
    # Find yesterday's currency supplies
    M, C, Mp, Cp = M / (1+gammam), C / (1 + gammac), Mp / (1 + gammam), Cp / (1+gammac)
    # Today's alpha/prices/welfare are yesterday's higher alpha/prices/value
    ah, phih, psih, Whp, Whohp = copy(al), copy(phi), copy(psi), copy(Wgrid[t]), copy(Whogrid[t])
    
# Graph prices    
figp,axp=plt.subplots()
axp.plot(tgrid,phigrid,label=r"$\phi$")
axp.plot(tgrid,psigrid,label=r"$\psi$")
axp.plot(tgrid,phisgrid,label=r"$\bar{\phi}$")
axp.plot(tgrid,psisgrid,label=r"$\bar{\psi}$")
lgd = plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc=2, bbox_to_anchor=(1.01, 1.025))
axp.set_title(f"{T} Period Prices")
axp.set_xlabel(r"$t$")
axp.set_ylabel(r"Prices")

plt.tight_layout()
add=r"\alltogether\\"+str(T)+"periodsp.pdf"
plt.savefig(file+add, bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.close()

# Quantities
figq,axq=plt.subplots() 
axq.plot(tgrid,qblgrid,label=r"$q_l^b$")
axq.plot(tgrid,qmlgrid,label=r"$q_l^m$")
axq.plot(tgrid,qbhgrid,label=r"$q_h^b$")
axq.plot(tgrid,qmhgrid,label=r"$q_h^m$")
axq.plot(tgrid,qhoblgrid,label=r"$\tilde{q}_l^b$")
axq.plot(tgrid,qhomlgrid,label=r"$\tilde{q}_l^m$")
axq.plot(tgrid,qhobhgrid,label=r"$\tilde{q}_h^b$")
axq.plot(tgrid,qhomhgrid,label=r"$\tilde{q}_h^m$")
axq.plot(tgrid,qsbgrid,label=r"$\bar{q}^b$")
axq.plot(tgrid,qsmgrid,label=r"$\bar{q}^m$")
lgd2 = plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc=2, bbox_to_anchor=(1.01, 1.025))
axq.set_ylim(0,1.05)
axq.set_xlabel(r"$t$")
axq.set_ylabel(r"Quantities")
axq.set_title(f"{T} Period Quantities")
add2=r"\alltogether\\"+str(T)+"periodsq.pdf"
plt.savefig(file+add2, bbox_extra_artists=(lgd2,), bbox_inches='tight')
plt.close()

# Real Balances
figrb,axrb=plt.subplots() 
axrb.plot(tgrid,rbmlgrid,label=r"$z_l^m$")
axrb.plot(tgrid,rbclgrid,label=r"$z_l^c$")
axrb.plot(tgrid,rbmhgrid,label=r"$z_h^m$")
axrb.plot(tgrid,rbchgrid,label=r"$z_h^c$")
axrb.plot(tgrid,rbmholgrid,label=r"$\tilde{z}_l^m$")
axrb.plot(tgrid,rbcholgrid,label=r"$\tilde{z}_l^c$")
axrb.plot(tgrid,rbmhohgrid,label=r"$\tilde{z}_h^m$")
axrb.plot(tgrid,rbchohgrid,label=r"$\tilde{z}_h^c$") 
axrb.plot(tgrid,rbmsgrid,label=r"$\bar{z}^m$")
axrb.plot(tgrid,rbcsgrid,label=r"$\bar{z}^c$")
lgd3 = plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc=2, bbox_to_anchor=(1.01, 1.025))
axrb.set_xlabel(r"$t$")
axrb.set_ylabel(r"Real Balances")
axrb.set_title(f"{T} Period Real Balances")
add3=r"\alltogether\\"+str(T)+"periodsrb.pdf"
plt.savefig(file+add3, bbox_extra_artists=(lgd3,), bbox_inches='tight')
plt.close()

# Currency Holdings
figch,axch=plt.subplots() 
axch.plot(tgrid,mgrid,label=r"$m$")
axch.plot(tgrid,cgrid,label=r"$c$")
axch.plot(tgrid,mhogrid,label=r"$\tilde{m}$")
axch.plot(tgrid,chogrid,label=r"$\tilde{c}$") 
axrb.plot(tgrid,msgrid,label=r"$\bar{m}$")
axrb.plot(tgrid,csgrid,label=r"$\bar{c}$")
lgd5 = plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc=2, bbox_to_anchor=(1.01, 1.025))
axch.set_xlabel(r"$t$")
axch.set_ylabel(r"Currency Holdings")
axch.set_title(f"{T} Period Currency Holdings")
add5=r"\alltogether\\"+str(T)+"periodsch.pdf"
plt.savefig(file+add5, bbox_extra_artists=(lgd5,), bbox_inches='tight')
plt.close()

# Welfare
figw,axw=plt.subplots() 
axw.plot(tgrid,Wgrid,label=r"$\mathbb{E}W$")
axw.plot(tgrid,Wlgrid,label=r"$W|\alpha=\alpha_l$")
axw.plot(tgrid,Whgrid,label=r"$W|\alpha=\alpha_h$")
axw.plot(tgrid,Whogrid,label=r"$\mathbb{E}\tilde{W}$ & $\tilde{W}|\alpha=\alpha_h$")
axw.plot(tgrid,Wholgrid,label=r"$\tilde{W}|\alpha=\alpha_l$")
axw.plot(tgrid,Wsgrid,label=r"$\bar{W}$")
lgd4 = plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc=2, bbox_to_anchor=(1.01, 1.025))
axw.set_title(f"{T} Period Welfare")
axw.set_xlabel(r"$t$")
axw.set_ylabel(r"$W$")

plt.tight_layout()
add4=r"\alltogether\\"+str(T)+"periodsw.pdf"
plt.savefig(file+add4, bbox_extra_artists=(lgd4,), bbox_inches='tight')
plt.close()

# Check prices are the same and program converged
figc,axc=plt.subplots(1,2) 
axc[0].plot(tgrid[:T],phidgrid,label=r"$\phi$")
axc[0].plot(tgrid[:T],psidgrid,label=r"$\psi$")
axc[0].legend()
axc[0].set_title(f"{T} Period Price Differential")
axc[0].set_xlabel(r"$t$")
axc[0].set_ylabel(r"Price Differential")
axc[1].plot(tgrid[:T],siggrid)
axc[1].set_title(f"{T} Period Convergance")
axc[1].set_xlabel(r"$t$")
axc[1].set_ylabel(r"Convergeance")
axc[1].set_ylim(-.05,1.05)

plt.tight_layout()
add5=r"\alltogether\\"+str(T)+"periodscheck.pdf"
plt.savefig(file+add5)
plt.close()

In [59]:
# Test these out
M = 1 # Original money supply
C = 1 # Original cc supply
pi = .1 # Chance tomorrow is low state
gammam = .02 # inflation of money
gammac = 0 # inflation of cc
mu = .99 # share of buyers (vs hodlers)
alphal = (gammac + 1 - B) / (gammam + 1 - B) + .05 # Slightly above indifference condition
alphah = (1 + alphal) / 2 # Don't want it to be 1, or money is not valued
Mp = M * (1 + gammam) # the money supply next period
Cp = C * (1 + gammac) # the cc supply next period
guessm, guessc = Mp / mu, Cp / mu
phil, psil = ststprices(alphal, Mp, Cp, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta) # prices in low stst
Wlp = ststW(alphal, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta)
Wholp = ststW(alphal, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta)
phih, psih = ststprices(alphah, Mp, Cp, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta) # prices in high stst
Whp = ststW(alphah, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta)
Whohp = ststW(alphah, gammam=gammam, gammac=gammac, lam=lam, theta=theta, eta=eta)

print(period_outcomes(M, C, pi, alphal, alphah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, guessm, guessc))
print(good_guess(M, C, pi, alphal, alphah, phil, psil, phih, psih, Wlp, Wholp, Whp, Whohp, mu, 151))

(1.4109749978428083, 1.4109749978428101, 1.4109749978428066, 0.19796138009590025, 0.19796138009590025, 0.19796138009590028, 1.0251845341221459, 0.50673112190755498, 0.5878561278916522, 41.802243338726392, 0.8450783181506148, 0.8227904754979041, 0.7949910480819471, 0.6799229210275765, 0.99999, 0.19092034166841548, 0.99999, 0.1469315229562492, 27.819794283118679, 27.985576296692127, 27.801374059388298, 22.482495851915019, 22.561495839547788, 22.482495851915019, True)
(1.0251845341221462, 0.58785612789164876)
